In [2]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import numpy as np

Using TensorFlow backend.


In [7]:
dir_name = 'dataset_to_train/'


def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []

    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "npz":
        
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:

            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'

            syntax = ' '.join(syntax.split())

            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

train_features, texts = load_data(dir_name)




In [8]:

# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
a=tokenizer.fit_on_texts([load_doc('Vocabulary/carbon_A+B+C.vocab')])

# Add one spot for the empty word in the vocabulary 
vocab_size = len(tokenizer.word_index) + 1
# Map the input sentences into the vocabulary indexes
train_sequences = tokenizer.texts_to_sequences(texts)
# The longest set of boostrap tokens
max_sequence = max(len(s) for s in train_sequences)
# Specify how many tokens to have in each input sentence
max_length = 17
print(tokenizer.word_index)
print(max_sequence)
print("##########################################################################################################################   ")
print(train_sequences)
def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_sequence)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            #print(out_seq)
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            #print(in_seq[-48:0])
            #print(out_seq)
            X.append(in_seq[-48:])
            y.append(out_seq)
        #print(X)    
    return np.array(X), np.array(y), np.array(image_data)
#print(train_sequences, train_features)
X, y, image_data = preprocess_data(train_sequences, train_features)



{',': 1, '{': 2, '}': 3, 'body': 4, 'accordionItem': 5, 'btn-primary': 6, 'btn-secondary': 7, 'form': 8, 'text-input-basic': 9, 'Radiobutton': 10, 'Checkbox': 11, 'Toggle': 12, 'search': 13, '<START>': 14, 'header': 15, '<END>': 16}
16
##########################################################################################################################   
[[14, 9, 1, 9, 1, 9, 1, 5, 1, 5, 1, 7, 3, 16], [14, 9, 1, 9, 1, 9, 1, 5, 1, 6, 3, 16], [14, 9, 1, 9, 1, 9, 1, 5, 7, 1, 6, 3, 16], [14, 9, 1, 9, 1, 9, 1, 9, 1, 5, 1, 6, 3, 16], [14, 9, 1, 9, 1, 9, 1, 9, 1, 5, 1, 7, 1, 6, 3, 16]]


In [8]:


image_model = Sequential()
image_model.add(Conv2D(16, (3, 3), padding='valid', activation='relu', input_shape=(256, 256, 3,)))
image_model.add(Conv2D(16, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(32, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
image_model.add(Conv2D(64, (3,3), activation='relu', padding='same', strides=2))
image_model.add(Conv2D(128, (3,3), activation='relu', padding='same'))

image_model.add(Flatten())
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))
image_model.add(Dense(1024, activation='relu'))
image_model.add(Dropout(0.3))

image_model.add(RepeatVector(max_length))

visual_input = Input(shape=(256, 256, 3,))
encoded_image = image_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)



decoder = concatenate([encoded_image, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)


model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:

filepath="A+B+C-org-weights-epoch-{epoch:04d}--val_loss-{val_loss:.4f}--loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=50)
callbacks_list = [checkpoint]

In [10]:
# Train the model
model.fit([image_data, X], y, batch_size=1, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=100)
model_json = model.to_json()
with open("A+B+C-100.json", "w") as json_file:
    json_file.write(model_json)

C:\Users\SourabhKulkarni\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 705 samples, validate on 79 samples
Epoch 1/100
 13/705 [..............................] - ETA: 43:26 - loss: 2.9542

KeyboardInterrupt: 